In [1]:
import os
import numpy as np
import pandas as pd
from scipy.signal import resample
from scipy.stats import mode

from config import Config

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [2]:
data_dir = "../../../data/dataset/raw/"
subjects = os.listdir(data_dir)
gestures = Config.GESTURES
learning_rate = 3e-4

test_subject = "007"

In [3]:
def __pre_process_recording(data: pd.DataFrame) -> pd.DataFrame:
    data.drop(columns=["time"], inplace=True)
    data.drop(0, inplace=True)  # Remove first All-0 row
    return data - data.iloc[:10].median()  # Initial position correction

In [4]:
train_dataset = pd.DataFrame()
test_dataset = pd.DataFrame()

for subject in subjects:
    for gesture in Config.GESTURES:
        gesture_dir = os.path.join(data_dir, subject, gesture)
        recordings = os.listdir(gesture_dir)
        for recording in recordings:
            file_path = os.path.join(gesture_dir, recording)
            data = pd.read_csv(file_path)
            # data.drop(columns=["time"], inplace=True)
            data = __pre_process_recording(data)
            data = data.apply(resample, args=(Config.SEGMENT_LEN, None, 0))
            
            for_training = True
            # subject != test_subject

            # ... Distance to thumb finger-tip
            data["drf0"] = np.sqrt(
                np.square(data["rf0x"] - data["rpx"]) +
                np.square(data["rf0z"] - data["rpz"])
            )

            # ... Distance to index finger-tip
            data["drf1"] = np.sqrt(
                np.square(data["rf1x"] - data["rpx"]) +
                np.square(data["rf1z"] - data["rpz"])
            )

            # ... Distance to middle finger-tip
            data["drf2"] = np.sqrt(
                np.square(data["rf2x"] - data["rpx"]) +
                np.square(data["rf2z"] - data["rpz"])
            )

            # ... Thumb coordineates wrt palm (for determining orientation)
            data["drf0x"] = data["rf0x"] - data["rpx"]
            data["drf0z"] = data["rf0z"] - data["rpz"]

            # Features v1.0
            # data["drf0x"] = data["rf0x"] - data["rpx"]
            # data["drf0y"] = data["rf0y"] - data["rpy"]
            # data["drf0z"] = data["rf0z"] - data["rpz"]

            # data["drf1x"] = data["rf1x"] - data["rpx"]
            # data["drf1y"] = data["rf1y"] - data["rpy"]
            # data["drf1z"] = data["rf1z"] - data["rpz"]

            data["label"] = Config.GESTURES.index(gesture)

            if for_training:
                train_dataset = pd.concat([train_dataset, data])
            else:
                test_dataset = pd.concat([test_dataset, data])

train_dataset.reset_index(inplace=True)
test_dataset.reset_index(inplace=True)
train_dataset

,index,rpx,rpy,rpz,lpx,lpy,lpz,rf0x,rf0y,rf0z,...,lf3z,lf4x,lf4y,lf4z,drf0,drf1,drf2,drf0x,drf0z,label
0,0,-0.279193,2.860625,0.180697,0.0,0.0,0.0,39.035673,5.143393,-1.376886,...,0.0,0.0,0.0,0.0,39.345708,10.635994,1.533212,39.314866,-1.557583,0
1,1,0.066689,-0.703069,-0.005151,0.0,0.0,0.0,-9.912681,-1.260604,0.290979,...,0.0,0.0,0.0,0.0,9.983763,2.689512,0.442822,-9.979370,0.296130,0
2,2,-0.038081,0.389705,0.013693,0.0,0.0,0.0,5.403865,0.705512,-0.180760,...,0.0,0.0,0.0,0.0,5.445420,1.475429,0.222900,5.441947,-0.194452,0
3,3,0.036351,-0.285972,-0.036925,0.0,0.0,0.0,-3.680618,-0.524906,0.150023,...,0.0,0.0,0.0,0.0,3.721668,1.018314,0.118617,-3.716970,0.186948,0
4,4,0.005604,0.141688,-0.056090,0.0,0.0,0.0,2.603108,0.243325,-0.069009,...,0.0,0.0,0.0,0.0,2.597537,0.682773,0.160472,2.597505,-0.012919,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173845,145,-12.579234,52.913419,24.163508,0.0,0.0,0.0,-10.636691,60.100964,28.265347,...,0.0,0.0,0.0,0.0,4.538563,6.571308,3.612464,1.942543,4.101839,13
173846,146,-11.945975,50.714738,23.003096,0.0,0.0,0.0,-10.166892,57.696697,27.071090,...,0.0,0.0,0.0,0.0,4.440012,5.988037,3.282502,1.779083,4.067994,13
173847,147,-12.396848,53.237922,24.189669,0.0,0.0,0.0,-10.599009,60.528799,28.190052,...,0.0,0.0,0.0,0.0,4.385805,6.197087,3.473064,1.797838,4.000383,13
173848,148,-11.439843,49.512851,22.361238,0.0,0.0,0.0,-9.779066,56.302485,26.153290,...,0.0,0.0,0.0,0.0,4.139788,5.483110,2.985410,1.660777,3.792052,13


In [5]:
diff_features = train_dataset[Config.DIFF_FEATURES]
dist_features = train_dataset[Config.DIST_FEATURES]

# dist_features = dist_features / dist_features.max(axis=0)

features = pd.concat([dist_features, diff_features], axis=1)
features

,drf0,drf1,drf2,drf0x,drf0z
0,39.345708,10.635994,1.533212,39.314866,-1.557583
1,9.983763,2.689512,0.442822,-9.979370,0.296130
2,5.445420,1.475429,0.222900,5.441947,-0.194452
3,3.721668,1.018314,0.118617,-3.716970,0.186948
4,2.597537,0.682773,0.160472,2.597505,-0.012919
...,...,...,...,...,...
173845,4.538563,6.571308,3.612464,1.942543,4.101839
173846,4.440012,5.988037,3.282502,1.779083,4.067994
173847,4.385805,6.197087,3.473064,1.797838,4.000383
173848,4.139788,5.483110,2.985410,1.660777,3.792052


# Scaler Metrics

In [20]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(features)
print(scaler.data_min_)
print(scaler.data_max_ - scaler.data_min_)

[   0.            0.            0.         -120.07078572  -85.7437317 ]
[  1.           1.           1.         304.62265771 177.02786573]


In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(features)

print(scaler.mean_)
print(np.sqrt(scaler.var_))

[41.12247478 29.60277426 43.97139651 36.31055241 -3.3004874 ]
[42.88667296 31.32058285 44.56781597 43.15776087 18.39518668]


In [ ]:
# ... Thumb coordineates wrt palm (for determining orientation)
    diff_features["drf0x"] = data["rf0x"] - data["rpx"]
    diff_features["drf0z"] = data["rf0z"] - data["rpz"]

In [ ]:
(features - scaler.mean_) / np.sqrt(scaler.var_) 

In [ ]:
train_features = train_dataset[Config.INFERENCE_FEATURES + ["label"]]
test_features = test_dataset[Config.INFERENCE_FEATURES + ["label"]]
train_features

In [ ]:
import tensorflow as tf

from tensorflow.keras import layers, models, losses, optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
scaler = MinMaxScaler()
# X = features.drop(columns=["label"]).to_numpy()
# X = scaler.fit_transform(X)
# X = X.reshape((-1, config.SEGMENT_LEN, len(config.INFERENCE_FEATURES)))
# y = features["label"].to_numpy().reshape((-1, config.SEGMENT_LEN))
# y, _ = mode(y, axis=1)

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.33, random_state=42
# )

X_train = train_features.drop(columns=["label"]).to_numpy()
X_test = test_features.drop(columns=["label"]).to_numpy()

scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

X_train = X_train.reshape((-1, config.SEGMENT_LEN, len(config.INFERENCE_FEATURES)))
X_test = X_test.reshape((-1, config.SEGMENT_LEN, len(config.INFERENCE_FEATURES)))

y_train = train_features["label"].to_numpy().reshape((-1, config.SEGMENT_LEN))
y_test = test_features["label"].to_numpy().reshape((-1, config.SEGMENT_LEN))

y_train, _ = mode(y_train, axis=1)
y_test, _ = mode(y_test, axis=1)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


In [ ]:
def conv_block_1d():
    inputs = layers.Input(shape=(config.SEGMENT_LEN, 1))
    x = layers.BatchNormalization()(inputs)
    x = layers.Conv1D(8, 3, activation="selu")(x)
    x = layers.Conv1D(8, 3, activation="selu")(x)
    x = layers.MaxPool1D(2)(x)
    x = layers.Conv1D(8, 3, activation="selu")(x)
    x = layers.Conv1D(8, 3, activation="selu")(x)
    x = layers.MaxPool1D(2)(x)
    x = layers.Conv1D(16, 3, activation="selu")(x)
    x = layers.Conv1D(16, 3, activation="selu")(x)
    x = layers.MaxPool1D(2)(x)
    x = layers.Conv1D(16, 3, activation="selu")(x)
    x = layers.Conv1D(16, 3, activation="selu")(x)
    x = layers.MaxPool1D(2)(x)
    x = layers.Flatten()(x)
    output = layers.Dense(64)(x)

    return inputs, output

def get_model(n_channels: int):
    inputs = []
    features = []

    for _ in range(n_channels):
        input_1d, features_1d = conv_block_1d()
        inputs.append(input_1d)
        features.append(features_1d)

    x = layers.concatenate(features, axis=-1)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(len(gestures), activation="softmax")(x)

    return models.Model(inputs, output)

In [ ]:
model = get_model(n_channels=len(config.INFERENCE_FEATURES))

loss = losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = optimizers.Adam(learning_rate=learning_rate)
model.compile(
    loss=loss,
    optimizer=optimizer,
    metrics=["accuracy"]
)

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=30,
        restore_best_weights=True
    )
]

history = model.fit(
    x=np.split(X_train, len(config.INFERENCE_FEATURES), axis=-1),
    y=y_train,
    validation_data=(
        np.split(X_test, len(config.INFERENCE_FEATURES), axis=-1),
        y_test
    ),
    batch_size=32,
    epochs=300,
    verbose=1,
    callbacks=callbacks
)

In [ ]:
model.evaluate(np.split(X_test, len(config.INFERENCE_FEATURES), axis=-1), y_test)

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams.update({
    # "text.usetex": True,
    "font.family": "serif",
    # "font.serif": ["Computer Modern Roman"],
    "font.size": 22,
    "text.color": "#212121",
    "axes.edgecolor": "#212121",
    "xtick.color": "#212121",
    "ytick.color": "#212121",
    "axes.labelcolor": "#212121",
    'legend.frameon': False,
})

fig = plt.figure(figsize=(8, 6))
ax = fig.gca()
ax.plot(history.history["loss"], "-", color="#212121", label="Train Loss")
ax.plot(history.history["val_loss"], "--", color="#212121", label="Validation Loss")
ax.set_xlabel("Epoch")
ax.set_ylabel("Loss")
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.title("Learning Curves")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# tf.keras.backend.clear_session()

In [ ]:
import joblib
joblib.dump(scaler, "../../models/scaler.joblib")
model.save("../../models/stack_cnn")